# Google

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# install with pip: transformers, sentencepiece 

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
input_text = "repeat the sentence 5 times: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_length=100)
print(tokenizer.decode(outputs[0]))

<pad> 5 times</s>


In [15]:
tokenizer.decode(outputs[0])

'<pad> 5 times</s>'

# LLaMa 2, 7B

In [6]:
import subprocess, time

In [7]:
def run(ckpt, task):
    """
    Runs a terminal command and prints the output followed by error message.
    Args:
        ckpt (str): Path to LLaMa model checkpoint
        task (str): Path to Python task
    Returns:
        float: Command execution time
    """
    start = time.time()
    cmd = f"torchrun --nproc_per_node 1 {task} --ckpt_dir {ckpt}/ --tokenizer_path tokenizer.model --max_seq_len 512 --max_batch_size 6"
    result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    end = time.time()
    exec = end-start
    print(f"Output:\n{result.stdout}")
    print(f"Error:\n{result.stderr}")
    print(f"Execution time: {exec} seconds")
    return exec 

In [8]:
run('../llama/llama-2-7b', '../llama/example_text_completion.py')

CalledProcessError: Command 'torchrun --nproc_per_node 1 ../llama/example_text_completion.py --ckpt_dir ../llama/llama-2-7b/ --tokenizer_path tokenizer.model --max_seq_len 512 --max_batch_size 6' returned non-zero exit status 1.